In [62]:
import numpy as np 
import pandas as pd


from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB

from sklearn.linear_model import LogisticRegression

from sklearn import tree
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier 

In [63]:
def read_data_from_csv(file_path: str) -> pd.DataFrame:
    return pd.read_csv(file_path)

In [113]:
REVIEWS_DATASET_PATH = './datasets/Womens_Clothing_E-Commerce_Reviews.csv'
reviews_dataset = read_data_from_csv(file_path=REVIEWS_DATASET_PATH)

## Resumen del dataset

El dataset contiene 23486 filas, cada una correspondiente a una reseña con unas 10 columnas correspondiente a variables explicadas a continuación.
### Descripción de cada columna

| Columna  | Descripción  |
|---|---|
| Clothing ID | Identificador numérico de la prenda especificada en la crítica |
| Age | Valor numérico correspondiente a la edad de la persona que realizo la crítica |
| Title | Titulo o encabezado de la crítica |
| Reviw Text | Contenido de la reseña realizada |
| Rating | Valor numérico en el rango del 1 al 5, siendo el 1 representante de máxima inconformidad y el 5 correspondiente al mejor valor posible |
| Recommended IND | Valor entre 0 y 1, representando respectivamente si el cliente no recomienda el producto o por el contrario si lo recomienda |
| Positive Feedback Count | Cantidad de reseñas positivas recibidas para el mismo producto |
| Division Name | Nombre de la división a la cual pertenece el producto |
| Department Name | Nombre del departamento al cual pertenece el producto |
| Class Name | Nombre de la clase al cual pertenece el producto |

### Categoria de variables

- Variables Cuantitativas : Age, Positive Feedback Count.

- Variables Cualitativas : Clothing ID, Title, Review Text, Rating, Recommended IND, Division Name, Department Name, Class Name.


In [65]:
reviews_dataset.dtypes

In [66]:
reviews_dataset

## Limpieza de Datos

### DATOS NULOS

Observamos la cantidad de nulos que poseen cada columna y el porcentaje que representan dicha cantidad de datos en cada columna

In [67]:
reviews_dataset.isna().sum()

In [68]:
reviews_dataset.isnull().sum() *100 / len(reviews_dataset.index)

Debido a que el trabajo a realizar con el dataset consiste en clasificar una reseña como una crítica positiva o una negativa a partir del texto en el campo “Review Text”, decidimos eliminar las columnas que no resultan de utilidad para dicho objetivo. De esta forma, obtenemos un dataset con las columnas necesarias para identificar las distintas prendas, el texto escrito en la reseña y por último la columna "Rating" para el desarrollo del ítem D, motivo por el cual conservamos dicha columna.

In [69]:
columns_eliminate= ['Unnamed: 0','Age','Title','Recommended IND','Positive Feedback Count','Division Name','Department Name','Class Name']
reviews_dataset.drop(columns_eliminate,axis = 'columns',inplace = True)

reviews_dataset

Por ultimo dado a que las filas con valores inválidos en la columna de "Review Text" son un total de 845 que representan una cantidad menor al 4% de la totalidad de los datos, optamos por eliminar dichas filas.

In [70]:
reviews_dataset.dropna(inplace=True)
reviews_dataset.shape

## Construccion de una nueva variable:

## Verificación:
Compruebo que la variable Rating no posea valores fuera de rango (1 al 5) antes de operar con dicha columna

In [71]:
reviews_dataset.Rating.value_counts()

In [72]:
reviews_dataset['Rating Category'] = reviews_dataset['Rating'].apply(lambda x: 'Negativo' if x <= 3 else 'Positivo')
reviews_dataset

In [73]:
reviews = reviews_dataset.loc[:, ['Review Text', 'Rating Category']]
reviews

In [74]:
le = LabelEncoder()
reviews['Rating Category'] = le.fit_transform(reviews['Rating Category'])
reviews

In [75]:
reviews.dtypes

In [76]:
stop_words = set(stopwords.words('english'))
vect = CountVectorizer(stop_words=stop_words)
vect_df = vect.fit_transform(reviews['Review Text'].values.tolist())
vect_df.shape

In [77]:
X_train, X_test, y_train, y_test = train_test_split(vect_df, reviews['Rating Category'], test_size=0.3, random_state=10)

### Naive Bayes

In [78]:
nb_clf = MultinomialNB()
nb_clf.fit(X_train, y_train)

In [79]:
y_nb_pred = nb_clf.predict(X_test)
report_NB = classification_report(y_test, y_nb_pred, digits=3)

### Regresión Logística

Para poder utilizar l función, Fit se deben verificar dos cosas:

i. Que no haya datos NaN

ii. Que cada elemento en la fila X_train, sea un vector.

In [80]:
lr = LogisticRegression(C=0.5, max_iter=150)
lr.fit(X_train, y_train)

In [81]:
y_lr_pred = lr.predict(X_test)
report_LR =classification_report(y_test, y_lr_pred, digits=3)

### Árboles de decisión

In [82]:
tree = tree.DecisionTreeClassifier(criterion='gini',min_samples_split=10,min_samples_leaf=2,ccp_alpha=0.0,max_depth=10) 

tree.fit(X_train,y_train)

In [83]:
y_tree_pred = tree.predict(X_test)

report_DT = classification_report(y_test, y_tree_pred, digits=3)

### Random Forest

In [84]:
forest_model= RandomForestClassifier(criterion='gini',min_samples_split=15,min_samples_leaf=7,ccp_alpha=0.0,max_depth=12, n_estimators=15)

forest_model.fit(X_train,y_train)

In [85]:
y_forest_pred = forest_model.predict(X_test)

report_RF = classification_report(y_test, y_forest_pred, digits=3)

## Aclaración:

En los últimos dos modelos utilizados, Árboles de decisión y Random Forest se puede implementar CrossValidation con el fin de encontrar los hiperparametros que mejoren el resultado obtenido según una determinada métrica. 

Específicamente se puede implementar un RandomSearch para el proceso de CrossValidaton y de esta forma probar una cierta cantidad de conjuntos aleatorios de hiperparametros pertenecientes a un rango que le brindamos al proceso. En su contraparte, también es posible utilizar GridSearch para probar todas las combinaciones posibles de hipeparametros entre los rangos brindados al proceso y obtener la mejor combinación.

Por ejemplo en el modelo de Random Forest es posible utilizar CrossValidation para los hipervalores: 'criterion', 'min_samples_split', 'min_samples_leaf', 'ccp_alpha' y 'max_depth'.

Sin embargo, preferimos no utilizar CrossValidation con el fin de obtener un mejor resultado a la hora de comparar los modelos, es decir, realizar la comparación con una de las posibilidades de hiperparametros y no con un conjunto que de mejores resultados.

### comparación de los modelos

In [96]:
print("Naive Bayes: \n", report_NB)
print("Regresión logística: \n",report_LR)
print("Árboles de decisión \n:",report_DT)
print("Random Forest: \n",report_RF)

### Implementación del mejor modelo:

Debido a que la métrica F1-score emplea los valores tanto de la precisión como del recall, optamos por darle mayor pero a dicha métrica a la hora de elegir el mejor modelo. Por este motivo optamos por utilizar Naive Bayes para catalogar a las reseñas en las 5 categorías de Rating. Dado a que no utilizaremos la columna de Rating binario “Rating Category” la eliminaremos.

In [102]:
reviews_dataset.drop(columns = 'Rating Category', inplace = True)
reviews_dataset

In [109]:
X_train, X_test, y_train, y_test = train_test_split(vect_df, reviews_dataset['Rating'], test_size=0.3, random_state=10)

nb_clf = MultinomialNB()
nb_clf.fit(X_train, y_train)

y_nb_pred = nb_clf.predict(X_test)

print(classification_report(y_test, y_nb_pred, digits=3))

### Conclusiones:



En primer lugar separamos nuevamente los datos del dataset filtrado en con una proporción del 70% y 30 % para posteriormente entrenar un modelo de Naive Bayes con el objetico de catalogar las reseñas entre las 5 categorías de Rating. 

Al analizar los resultados de las métricas en las distintas categorías, podemos observar que son bastante distintas. Por ejemplo, observamos que la metrica F1 vale 0.074 para la categoría de reseñas con 1 estrella pero sin embargo vale 0.810 para aquellas que poseen 5 estrellas. Una posible explicación para esta situación es la cantidad de datos que poseen cada categoría, ya que la categoría de 5 estrellas cuenta con 12540 reseñas contra las 821 reseñas categorizadas con 1 estrella. 
En caso de desear un mejor resultado en las categorías donde peor se desempeña el modelo, se deberán utilizar otras estrategias como por ejemplo modificar los datos del dataset (emparejando las cantidades sea con datos nuevos u operado con los datos existentes), otra opción sería el control de los conjuntos de entrenamiento del modelo para que los mismos sean balaceados.  
